# TPP Advanced Visualization Dashboard

Interactive and advanced visualizations for TPP monitoring data analysis.

**Contents:**
1. Interactive plotly dashboards
2. Heatmaps and correlation matrices
3. Time-series decomposition
4. Distribution analysis
5. Multi-dimensional visualizations
6. Export ready-to-publish figures

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configure static plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print("✓ Visualization libraries loaded")

In [ ]:
# Load all sample data
with open('../sample-data/heart-rate-sample.json', 'r') as f:
    data_json = json.load(f)
df_json = pd.DataFrame(data_json)

with open('../sample-data/night-monitoring.json', 'r') as f:
    night_data = json.load(f)
df_night = pd.DataFrame(night_data)

df_csv = pd.read_csv('../sample-data/heart-rate-sample.csv')

# Combine and process
df = pd.concat([df_csv, df_json, df_night], ignore_index=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# Add time features
df['hour'] = df['timestamp'].dt.hour
df['date'] = df['timestamp'].dt.date

print(f"✓ Loaded {len(df)} records")
df.head()

## 1. Interactive Time-Series Dashboard

In [ ]:
# Create interactive plotly dashboard
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Heart Rate Over Time', 'Heart Rate Variability', 'Activity Pattern'),
    vertical_spacing=0.1,
    row_heights=[0.4, 0.3, 0.3]
)

# Heart Rate plot
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['heartRate'], 
              mode='lines+markers', name='Heart Rate',
              line=dict(color='red', width=2),
              marker=dict(size=6)),
    row=1, col=1
)

# Add mean line
mean_hr = df['heartRate'].mean()
fig.add_hline(y=mean_hr, line_dash="dash", line_color="darkred", 
              annotation_text=f"Mean: {mean_hr:.1f} bpm", row=1, col=1)

# HRV plot
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['hrv'],
              mode='lines+markers', name='HRV',
              line=dict(color='green', width=2),
              marker=dict(size=6)),
    row=2, col=1
)

# Activity pattern (categorical bar)
activity_colors = {'resting': 'blue', 'walking': 'orange', 'exercise': 'red', 'sleeping': 'purple'}
for activity in df['activity'].unique():
    activity_df = df[df['activity'] == activity]
    fig.add_trace(
        go.Scatter(x=activity_df['timestamp'], y=activity_df['heartRate'],
                  mode='markers', name=activity,
                  marker=dict(size=10, color=activity_colors.get(activity, 'gray'))),
        row=3, col=1
    )

# Update layout
fig.update_xaxes(title_text="Time", row=3, col=1)
fig.update_yaxes(title_text="Heart Rate (bpm)", row=1, col=1)
fig.update_yaxes(title_text="HRV (ms)", row=2, col=1)
fig.update_yaxes(title_text="Heart Rate (bpm)", row=3, col=1)

fig.update_layout(
    title_text="TPP Monitoring Interactive Dashboard",
    title_font_size=20,
    height=900,
    showlegend=True,
    hovermode='x unified'
)

fig.show()

## 2. Heatmap Visualizations

In [ ]:
# Create hour-of-day vs activity heatmap
pivot_hr = df.pivot_table(values='heartRate', index='activity', columns='hour', aggfunc='mean')

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Heart Rate heatmap
sns.heatmap(pivot_hr, annot=True, fmt='.1f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Heart Rate (bpm)'}, ax=axes[0])
axes[0].set_title('Average Heart Rate by Activity and Hour', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Activity Type')

# HRV heatmap
pivot_hrv = df.pivot_table(values='hrv', index='activity', columns='hour', aggfunc='mean')
sns.heatmap(pivot_hrv, annot=True, fmt='.1f', cmap='YlGnBu',
            cbar_kws={'label': 'HRV (ms)'}, ax=axes[1])
axes[1].set_title('Average HRV by Activity and Hour', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Hour of Day')
axes[1].set_ylabel('Activity Type')

plt.tight_layout()
plt.show()

## 3. Distribution and Density Plots

In [ ]:
# Multi-panel distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram with KDE
axes[0, 0].hist(df['heartRate'], bins=20, alpha=0.7, color='steelblue', edgecolor='black', density=True)
df['heartRate'].plot(kind='kde', ax=axes[0, 0], color='red', linewidth=2)
axes[0, 0].set_title('Heart Rate Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Heart Rate (bpm)')
axes[0, 0].set_ylabel('Density')
axes[0, 0].grid(True, alpha=0.3)

# Violin plot by activity
df.boxplot(column='heartRate', by='activity', ax=axes[0, 1])
axes[0, 1].set_title('Heart Rate Distribution by Activity', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Activity')
axes[0, 1].set_ylabel('Heart Rate (bpm)')
plt.sca(axes[0, 1])
plt.xticks(rotation=45)

# 2D density plot
axes[1, 0].hexbin(df['hrv'], df['heartRate'], gridsize=15, cmap='YlOrRd')
axes[1, 0].set_xlabel('HRV (ms)', fontsize=12)
axes[1, 0].set_ylabel('Heart Rate (bpm)', fontsize=12)
axes[1, 0].set_title('2D Density: HRV vs Heart Rate', fontsize=14, fontweight='bold')

# Q-Q plot for normality check
from scipy import stats as sp_stats
sp_stats.probplot(df['heartRate'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot: Heart Rate Normality', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 3D Interactive Visualization

In [ ]:
# 3D scatter plot: Time, HR, HRV
fig_3d = go.Figure(data=[go.Scatter3d(
    x=df['hour'],
    y=df['heartRate'],
    z=df['hrv'],
    mode='markers',
    marker=dict(
        size=8,
        color=df['heartRate'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="HR (bpm)")
    ),
    text=df['activity'],
    hovertemplate='<b>Hour:</b> %{x}<br>' +
                  '<b>HR:</b> %{y} bpm<br>' +
                  '<b>HRV:</b> %{z} ms<br>' +
                  '<b>Activity:</b> %{text}<extra></extra>'
)])

fig_3d.update_layout(
    title='3D Visualization: Time, Heart Rate, and HRV',
    scene=dict(
        xaxis_title='Hour of Day',
        yaxis_title='Heart Rate (bpm)',
        zaxis_title='HRV (ms)',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.3))
    ),
    height=700
)

fig_3d.show()